In [ ]:
import pandas as pd

In [ ]:
pd.set_option("mode.chained_assignment", None)

In [ ]:
from futil import * 

In [ ]:
wdi_df, series_df = load_wdi()

In [ ]:
series_df.columns

In [ ]:
series_df[series_df['Indicator Name'].str.contains('Primary government')][['Indicator Name', 'Long definition']].iloc[0].to_dict()

In [ ]:
def extract_indic_name_definition(string_fragment, columns=['Series Code', 'Indicator Name', 'Long definition']):
    xdf = series_df[series_df['Indicator Name'].str.contains(string_fragment)]
    print(xdf[columns].to_string())

In [ ]:
extract_indic_name_definition('budget')

In [ ]:
extract_indic_name_definition('Expense')

In [ ]:
extract_indic_name_definition('Tax revenue')

To use for government budget, options:

* Expense. GC.XPN.TOTL.CN (or GC.XPN.TOTL.ZS) for total expenses. But only current LCU.
* Tax revenue. GC.TAX.TOTL.CN (or GC.TAX.TOTL.GD.ZS) for total revenue. Also only current LCU

In [ ]:
project_df = load_projects()
project_df.head()

In [ ]:
project_df.wb_lendingproject_cost.hist()

In [ ]:
len(project_df[project_df.wb_lendingproject_cost.notna()])

In [ ]:
wb_df = project_df[project_df.donor_name == 'WB']
wb_df.groupby('wb_lendinginstrumenttype').agg(avg_size=('wb_lendingproject_cost', 'mean'))

In [ ]:
wb_df['start_date'] = pd.to_datetime(wb_df.start_date)
wb_df['created_year'] = wb_df.start_date.dt.year
wb_df[wb_df['created_year'] > 1980].groupby('created_year').agg(avg_size=('wb_lendingproject_cost', 'mean'))

* Looks pretty clearly like it's USD at period, not now, so will have to combine

So use:

* GDP at current USD, from WDI
* Multiply by % of GDP
* Compare to WB loan size

In [ ]:
relevant_codes = {
    'GDP_current_USD': 'NY.GDP.MKTP.CD',
    'GovtExpenses_current_USD': 'GC.XPN.TOTL.GD.ZS'
}

In [ ]:
wb_extract = extract_indicators(wdi_df, relevant_codes.values())

In [ ]:
def obtain_country_exp_time(expense_df, country_code):
    country_code

In [ ]:
expense_df = wb_extract[wb_extract['Indicator Code'] == 'GC.XPN.TOTL.GD.ZS']

In [ ]:
country_code = 'ZWE' # use Zim
country_exp_row = expense_df[expense_df['Country Code'] == country_code]
country_exp_row

In [ ]:
edf = country_exp_row.drop(columns=['Indicator Name', 'Indicator Code', 'Unnamed: 65'], errors='ignore').melt(id_vars=['Country Code', 'Country Name'])

In [ ]:
edf = edf.rename(columns = { 'variable': 'year' })

In [ ]:
edf.head()

In [ ]:
edf['year'] = pd.to_datetime(edf['year'])

In [ ]:
edf = edf.set_index('year')

In [ ]:
edf.interpolate()

In [ ]:
edf.plot.line()

In [ ]:
edf.head(n=20)

In [ ]:
edf2 = expense_df.drop(columns=['Indicator Name', 'Indicator Code', 'Unnamed: 65'], errors='ignore').melt(id_vars=['Country Code', 'Country Name'])

In [ ]:
edf2.head()

In [ ]:
len(edf2)

In [ ]:
edf2 = edf2[edf2['Country Code'].isin(wb_df.country_code)]

In [ ]:
print("Ratio not available: ",  edf2.value.isna().sum() / len(edf2))

In [ ]:
wb_df.created_year.hist()

In [ ]:
def calculate_ratio_na(indicator_code, earliest_year=1970):
    indic_extract = extract_indicators(wdi_df, [indicator_code])
    tdf = indic_extract.drop(columns=['Indicator Name', 'Indicator Code', 'Unnamed: 65'], errors='ignore').melt(id_vars=['Country Code', 'Country Name'])
    tdf['year'] = pd.to_datetime(tdf['variable']).dt.year
    tdf = tdf[tdf['year'] > earliest_year]
    tdf = tdf[tdf['Country Code'].isin(wb_df.country_code)]
    return tdf.value.isna().sum() / len(tdf)

In [ ]:
calculate_ratio_na('GC.XPN.TOTL.GD.ZS')

In [ ]:
calculate_ratio_na('DT.ODA.ODAT.GN.ZS')

In [ ]:
# Expense. GC.XPN.TOTL.CN (or GC.XPN.TOTL.ZS) for total expenses. But only current LCU.
# Tax revenue. GC.TAX.TOTL.CN (or GC.TAX.TOTL.GD.ZS) for total revenue. Also only current LCU

In [ ]:
candidate_vars = [
    'NY.GDP.MKTP.CD',
    'GC.XPN.TOTL.GD.ZS',
    'GC.XPN.TOTL.ZS',
    'GC.TAX.TOTL.CN',
    'GC.TAX.TOTL.GD.ZS',
    'IQ.CPA.FINQ.XQ',
    'SE.XPD.TOTL.GD.ZS',
    'GB.XPD.RSDV.GD.ZS',
    'DT.ODA.ODAT.GN.ZS'
]

In [ ]:
na_ratios = [calculate_ratio_na(code) for code in candidate_vars]
print(dict(zip(candidate_vars, na_ratios)))

### Result: at present, looks like we have to use just GDP in current USD. Going to have to just go with GDP.

**Note**: Weirdly, have more data points for education share of expenditure

In [ ]:
extract_indic_name_definition('GDP')

## Checking feasibility of windows with no active project

In [ ]:
project_df.head()

In [ ]:
project_df.crs_purpose_sector.value_counts()

In [ ]:
project_df.project_duration.hist()

In [ ]:
project_df.mmg_purpose_sectorname.value_counts()

In [ ]:
project_df.mmg_purpose_sector.value_counts()

In [ ]:
project_df.aiddata_sectorname.value_counts()

* **Note**: Will need to cross-match to SDGs, in order to match properly to data

In [ ]:
project_df['start_dt'] = pd.to_datetime(project_df['start_date'], format='%d%b%Y', errors='coerce')

In [ ]:
project_df.groupby('aiddata_sectorname').agg(
    number_projects=('ppd_project_id', 'nunique'),
    earliest_year=('start_dt', 'min'),
    latest_project=('start_dt', 'max'),
    number_donors=('donor_name', 'nunique'),
    avg_rating=('six_overall_rating', 'mean')
).to_csv('../data/aiddata_purposes_honig_summary_stats.csv')

In [ ]:
project_df.project_duration.describe()

In [ ]:
project_df['project_years'] = project_df['project_duration'] / 365

In [ ]:
project_df.project_years.describe()

In [ ]:
ccode = 'GUY'

In [ ]:
time_cols = ['country_code', 'donor_name', 'aiddata_sectorname', 'six_overall_rating', 'start_date', 'completion_date', 'project_duration']

In [ ]:
pdf = narrow_convert_project_data(project_df, time_cols)

In [ ]:
pdf[pdf.country_code == ccode]

In [ ]:
from datetime import datetime

In [ ]:
pdf.completion_dt.dt.month.hist()

### NB reasoning on years

1. Vast majority of projects have completion date as 1 January. Suggesting specific completion dates are generally not recorded, i.e., assume some normal distribution throughout year.
2. On the other hand, by project completion most activities should have been underway for some time. So it is plausible to consider the year of the project as year 1 for lags. If a project has completion date 2010, outcome effects on a lag of 5 years would be evaluated in 2014.
3. That also implies a no-treatment year is any in which no project is active and is not within the lag period of a completed project.

In [ ]:
from datetime import datetime, timedelta

In [ ]:
example_year = datetime(2000, 1, 1)

In [ ]:
len(pdf[pdf.start_dt > example_year])

In [ ]:
len(pdf)

In [ ]:
pdf.aiddata_sectorname.nunique()

In [ ]:
pdf.start_year.hist()

In [ ]:
active_projects = pdf[(pdf.start_dt < example_year) & (pdf.completion_dt > example_year)]

In [ ]:
active_projects.head()

In [ ]:
len(active_projects)

In [ ]:
example_lag_years = 5

In [ ]:
active_projects = pdf[(pdf.start_dt < example_year) & ((pdf.completion_dt + timedelta(days=(5*365))) > example_year)]

In [ ]:
len(active_projects)

In [ ]:
active_projects.country_code.nunique()

In [ ]:
sectors = active_projects.groupby('country_code')['aiddata_sectorname'].nunique()

In [ ]:
sectors

In [ ]:
sectors.ge(0).sum()

In [ ]:
active_projects.start_year.hist()

In [ ]:
active_projects.end_year.hist()

In [ ]:
pdf.country_code.nunique()

In [ ]:
total_countries = pdf.country_code.nunique()
earliest_year = pdf.start_year.min()
latest_year = pdf.start_year.max()

In [ ]:
year_range = range(int(earliest_year), int(latest_year))

In [ ]:
len(pdf.sector)

In [ ]:
def count_countries_in_window(year_lag=5, above_sector_count=0):
    countries_with_active_projects = []
    for year in year_range:
        ref_year = datetime(year, 1, 1)
        active_plus_lag_mask = (pdf.start_dt < ref_year) & ((pdf.completion_dt + timedelta(days=(5*365))) > ref_year)
        active_plus_lag = pdf[active_plus_lag_mask]
        with_sectors = active_plus_lag.groupby('country_code')['aiddata_sectorname'].nunique()
        countries_with_active_projects.append(with_sectors.ge(above_sector_count).sum())
    
    prop_countries_untreated = [1 - count / total_countries for count in countries_with_active_projects]
    return prop_countries_untreated

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ax.plot(year_range, countries_with_active_projects)

In [ ]:
fig, ax = plt.subplots()
ax.plot(year_range, count_countries_in_window())
ax.set_ylim([0, 1])

In [ ]:
fig, ax = plt.subplots()
ax.plot(year_range, count_countries_in_window(year_lag=5, above_sector_count=3))
ax.set_ylim([0, 1])

** In other words: can only get to binary using sectors